# Como realizar el analisis de datos en Jupyter 

[PyCon Ar en PyVideo.org](https://pyvideo.org/events/pycon-ar-2011.html)

- Explorar los datos

In [19]:
import pathlib

carpeta_datos = pathlib.Path('./datos')

In [33]:
archivos = [ elemento for elemento in carpeta_datos.glob('*/videos/*.json') ]

In [34]:
len(archivos)

237

In [35]:
{archivo.suffix for archivo in archivos }

{'.json'}

In [31]:
archivos[0].suffix

''

- Importar los datos con pandas

In [36]:
import pandas

In [58]:
def safe_df(archivo):
    try:
        df = pandas.read_json(archivo,typ='series')
    except ValueError as err:
        df = pandas.DataFrame()
        print('Fallo al cargar: ', archivo)
    return df

In [59]:
series = [ safe_df(archivo) for archivo in archivos ]

In [69]:
df = pandas.concat(series,axis=1).T

In [73]:
speakers = df.speakers.explode()

In [82]:
import bqplot.pyplot as plt

pandas.set_option('plotting.backend', 'bqplot.pyplot')

In [83]:
df.speakers.apply(len).sort_values().plot()
plt.show()

In [115]:
x=df.speakers.explode().value_counts()
plt.figure()
plt.bar(x)
plt.show()

TypeError: bar() missing 1 required positional argument: 'y'

In [120]:
x.bar()

AttributeError: 'Series' object has no attribute 'bar'

- Realizar graficos